In [2]:
import json
from utils import *
import re

In [3]:
sdi = SchoolDataIndex()

In [4]:
list(sdi.school_ids())[0]

'1010720019'

In [5]:
school_id = '1010720019'
sd = load_school_data(school_id)

In [6]:
soup = load_soup(sd['building'])

In [7]:
for comment in soup.children: break
url = re.findall('url: (.*)\n', comment)
parent_url = re.sub('[^/]*$', '', url[0])

In [16]:
building_data = []
image_id = 0
for row in soup.find_all('div', attrs={'class': 'row'}):
    cols = row.find_all('div', attrs={'class': 'col-md-4'})
    for col in cols:
        table = col.find('table')
        if table is None:
            continue
        heading = table.find('th')
        building_details = {} 
        images = col.find_all('img', attrs={'src': True})
        if images is not None and len(images) > 0:
            building_details['images'] = []

        image_dir = SCRAPED_FILE_DIRS['building'] + '/images'
        if not is_path_existed(image_dir):
            makedirs(image_dir)
        for img in images:
            img_url = parent_url + img['src']
            image_path = image_dir + '/' + school_id + '_' + '{:02}'.format(image_id) + '.jpg'
            image_id += 1
            with open(image_path, 'wb') as im_file:
                image = requests.get(img_url).content
                im_file.write(image)

            building_details['images'].append({
                'image_url': img_url,
                'image_path': image_path
            })


        for tab_row in table.find_all('td'):
            if tab_row.find('a') is None:
                key, val = re.sub('\s+', ' ', tab_row.text.strip()).split(':', maxsplit=1)
                building_details.update({key.strip(): val.strip()})

        building_data.append({
            'name': heading.text,
            'details': building_details
        })

In [17]:
building_data

[{'name': 'อาคารเรียน ลำดับที่ 1 ',
  'details': {'images': [{'image_url': 'https://bobec.bopp-obec.info/b01/7200191012124970120201029105049505747918.jpg',
     'image_path': '2022-09/html/school/building/images/1010720019_00.jpg'},
    {'image_url': 'https://bobec.bopp-obec.info/b01/7200191012124970120201029105116173283977.jpg',
     'image_path': '2022-09/html/school/building/images/1010720019_01.jpg'}],
   'แบบ': '004',
   'จำนวนห้องตามแปลน': '18 ห้อง',
   'สร้าง': 'พ.ศ. 2497',
   'สภาพปัจจุบัน': 'ดี'}},
 {'name': 'อาคารเรียน ลำดับที่ 2 ',
  'details': {'images': [{'image_url': 'https://bobec.bopp-obec.info/b01/720019201212527012020102910545160439057.jpg',
     'image_path': '2022-09/html/school/building/images/1010720019_02.jpg'},
    {'image_url': 'https://bobec.bopp-obec.info/b01/7200192012125270120201029105547936353701.jpg',
     'image_path': '2022-09/html/school/building/images/1010720019_03.jpg'}],
   'แบบ': '108 ล,108 ,ล/30,108 หรือ108 ชื่ออื่นๆ',
   'จำนวนห้องตามแปลน': '15 